In [1]:
import numpy.linalg as nl
import numpy as np

import scipy.io as sio

mat_contents = sio.loadmat('polynomial_regression_samples.mat')
mat_contents

{'__globals__': [],
 '__header__': b'MATLAB 5.0 MAT-file, Platform: MACI64, Created on: Fri Sep  1 15:17:33 2017',
 '__version__': '1.0',
 'x': array([[-0.0963728 , -0.08653028, -0.0311563 , -0.04982882, -0.06423248],
        [-0.01914529,  0.04967885,  0.00363335, -0.04549159,  0.05396517],
        [ 0.08841612,  0.02585844, -0.00017123, -0.04069409, -0.09145454],
        ..., 
        [-0.02372765,  0.05880506,  0.06872046, -0.08455554, -0.06840088],
        [ 0.05271218, -0.02336013, -0.06770073, -0.09082807,  0.05144121],
        [-0.05857526, -0.05709096, -0.02838865,  0.01909572, -0.04962651]]),
 'y': array([[ 0.95170245],
        [ 1.23652314],
        [-0.34383271],
        ..., 
        [ 0.99606357],
        [ 1.13095857],
        [ 1.09124217]])}

In [2]:
x_train = mat_contents['x']
a = np.ones((100000,1))

x_train = np.hstack((a,x_train))
print(x_train.shape)
x1 = x_train[0:25000]
x2 = x_train[25000:50000]
x3 = x_train[50000:75000]
x4 = x_train[75000:]
x_train1 = np.concatenate([x1,x2,x3])
x_test1 = x4

x_train2 = np.concatenate([x1,x2,x4])
x_test2 = x3

x_train3 = np.concatenate([x1,x3,x4])
x_test3 = x2

x_train4 = np.concatenate([x2,x3,x4])
x_test4 = x1

x_train_fold = [x_train1, x_train2, x_train3, x_train4]
x_test_fold = [x_test1, x_test2, x_test3, x_test4]


(100000, 6)


In [3]:
y_train = mat_contents['y']
y1 = y_train[0:25000]
y2 = y_train[25000:50000]
y3 = y_train[50000:75000]
y4 = y_train[75000:]

y_train1 = np.concatenate([y1,y2,y3])
y_test1 = y4

y_train2 = np.concatenate([y1,y2,y4])
y_test2 = y3

y_train3 = np.concatenate([y1,y3,y4])
y_test3 = y2

y_train4 = np.concatenate([y2,y3,y4])
y_test4 = y1

y_train_fold = [y_train1, y_train2, y_train3, y_train4]
y_test_fold = [y_test1, y_test2, y_test3, y_test4]


In [5]:
import itertools

def poly_mv_features(X_train, D):
    n = X_train.shape[0]
    columns = len(list(itertools.combinations_with_replacement([0,1,2,3,4,5], D)))
    X = np.empty([n, columns], dtype=float)
    print(X.shape, X_train.shape)
    combinations =  list(itertools.combinations_with_replacement([0,1,2,3,4,5], D))
    for row_num in range(n):
        for i,combi in enumerate(combinations):
            column_value = 1
            for column_num in combi:
                #print(column_num)
                column_value *= X_train[row_num][column_num]
            X[row_num][i] = column_value   
    return X

def error(X, y_true, weights):
    y_predicted = X.dot(weights)
    error = nl.norm(y_true-y_predicted) ** 2
    return error

def ridge(X, y, l):
    sizeidentity = X.shape[1]
    Xt = np.transpose(X)
    w = nl.inv(Xt.dot(X) + l*np.identity(sizeidentity)).dot(Xt).dot(y)
    return w

error_terms = np.empty([4, 4], dtype=float)
degree = []
weights = []
n = x_train.shape[0]
for d in range(1, 5):
    for l in range(1,5):
        lmd = 0.05*l
        total_error = 0
        for x_trainf,x_testf,y_trainf,y_testf in zip(x_train_fold, x_test_fold, y_train_fold, y_test_fold):        
            X = poly_mv_features(x_trainf, d)
            wt = ridge(X, y_trainf, lmd)
            weights.append(wt)
            X_testf = poly_mv_features(x_testf, d)
            err = error(X_testf, y_testf, wt)/25000
            print(d, err)
            total_error += err
        error_terms[d-1][l-1] = total_error/4
    

(75000, 6) (75000, 6)
(25000, 6) (25000, 6)
1 0.0588532322718
(75000, 6) (75000, 6)
(25000, 6) (25000, 6)
1 0.0566857669761
(75000, 6) (75000, 6)
(25000, 6) (25000, 6)
1 0.0574762167917
(75000, 6) (75000, 6)
(25000, 6) (25000, 6)
1 0.06128352542
(75000, 6) (75000, 6)
(25000, 6) (25000, 6)
1 0.0588531945865
(75000, 6) (75000, 6)
(25000, 6) (25000, 6)
1 0.0566856932332
(75000, 6) (75000, 6)
(25000, 6) (25000, 6)
1 0.0574762117211
(75000, 6) (75000, 6)
(25000, 6) (25000, 6)
1 0.0612836428783
(75000, 6) (75000, 6)
(25000, 6) (25000, 6)
1 0.0588531579954
(75000, 6) (75000, 6)
(25000, 6) (25000, 6)
1 0.056685620589
(75000, 6) (75000, 6)
(25000, 6) (25000, 6)
1 0.0574762077129
(75000, 6) (75000, 6)
(25000, 6) (25000, 6)
1 0.0612837613274
(75000, 6) (75000, 6)
(25000, 6) (25000, 6)
1 0.0588531224974
(75000, 6) (75000, 6)
(25000, 6) (25000, 6)
1 0.0566855490421
(75000, 6) (75000, 6)
(25000, 6) (25000, 6)
1 0.0574762047657
(75000, 6) (75000, 6)
(25000, 6) (25000, 6)
1 0.0612838807661
(75000, 21)

In [8]:
error_terms
#the smallest error term is  0.05852006 which corresponds to lambda = 0.15 and d = 3 or 2

array([[ 0.05857469,  0.05857469,  0.05857469,  0.05857469],
       [ 0.05852113,  0.05852039,  0.05852011,  0.05852016],
       [ 0.05852103,  0.05852032,  0.05852006,  0.05852013],
       [ 0.05852102,  0.05852032,  0.05852006,  0.05852012]])